# 作業場所をGCP上に構築

## ユーザ名を識別子として保存

In [ ]:
pwd = !pwd
username = pwd[0].split("/")[2]
username

## ssh鍵の準備

In [ ]:
!mkdir -p ~/.ssh
!rm -fv ~/.ssh/sshkey
!rm -fv ~/.ssh/sshkey.pub
!ssh-keygen -t rsa -C $username -f ~/.ssh/sshkey -q -N ""
!chmod 600 ~/.ssh/sshkey
r = !cat ~/.ssh/sshkey.pub
sshkeypub = r[0]
!ls -la ~/.ssh/

## 認証

In [ ]:
!gcloud auth activate-service-account --key-file topse-nagaku-4052cd1840ff.json

## デフォルトのリージョンとゾーンを設定

In [ ]:
!gcloud config set compute/zone us-west1-b
!gcloud config set compute/region us-west1

## gcloudコマンドの動作確認

In [ ]:
!gcloud compute instances list

In [ ]:
!gcloud compute images list

## 仮想マシンの起動

In [ ]:
r = !gcloud compute instances create $username-target \
--machine-type=n1-standard-2 --metadata ssh-keys="$username:$sshkeypub" \
--preemptible --tags=http-server --image-family=centos-7 \
--image-project=centos-cloud
print(r)
linux = r[2].split()
host = linux[4]
host

## 仮想マシンにansibleでdocker環境を作る

In [ ]:
ansible_ssh_private_key_file = "~/.ssh/sshkey"

with open('hosts', mode='w') as f:
    f.write(host + ' ansible_user=' + username + ' ansible_ssh_private_key_file='
            + ansible_ssh_private_key_file)

作成した接続情報を使う前に、対象となるLinuxマシンを信頼できるホストとして、fingerprint登録しておきます。JupyterのTerminalを使って、ダミーのssh実行を行うと登録されます。

In [ ]:
print("$ ssh -i "+ansible_ssh_private_key_file+" "+username+"@"+host)

上記のコマンドを実行し
```
（中略）
Are you sure you want to continue connecting (yes/no)? yes←信頼しますよの意味で入力する
（以下略）
```
として、fingerprint登録します。ssh接続は
```
exit
```
で終了させてください。

ansible経由でdocker環境を構築します。

In [ ]:
!ansible -i hosts $host -m yum -a "name=* state=latest" --become

In [ ]:
!ansible -i hosts $host -m yum -a "name=docker,docker-compose state=latest" --become

In [ ]:
!ansible -i hosts $host -m service \
-a "name=docker state=started enabled=yes" --become

dockerコマンド、docker-composeコマンドが動くことを確認します。

In [ ]:
!ansible -i hosts $host -a "docker ps -a" --become
!ansible -i hosts $host -a "docker-compose --version" --become

## sshカーネル用の設定ファイルを作成

In [ ]:
!echo "Host $username-target" > ~/.ssh/config
!echo "  Hostname $host" >> ~/.ssh/config
!echo "  User $username" >> ~/.ssh/config
!echo "  Port 22" >> ~/.ssh/config
!echo "  IdentityFile $ansible_ssh_private_key_file" >> ~/.ssh/config
!cat ~/.ssh/config